## 5.0 Импорты библиотек

In [1]:
import os
import yaml
import logging
import numpy as np
import scipy.stats as stats
import sys
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import levene
from typing import List, Any, Optional, Tuple, Dict
from datetime import datetime
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
from scipy.stats import pearsonr, spearmanr, kurtosis, skew
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from sklearn.base import BaseEstimator, TransformerMixin

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
# расширяем поле ноутбука для удобства
from IPython.display import display, HTML
display(HTML('<style>.container {width:87% !important;}</style>'))
display(HTML("<style>.output_scroll {height:auto !important; max-height:10000px !important;}</style>"))

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
# Настройки для pandas (количество отображаемых колонок)
pd.set_option('display.max_columns', 100)

In [5]:
# Определение стиля для pyplot
plt.style.use('ggplot')

In [6]:
# Текущая рабочая директория
cwd = Path().resolve()

# Поднимаемся на один уровень выше
project_root = cwd.parent

# Добавляем корень проекта в sys.path
sys.path.append(str(project_root))

# Загрузка данных из config.yaml
from src.data import downloader, loader, preprocessor, saving
from src.features import feat_preprocessing

# Путь к файлу config.yaml
config_path = project_root / "config" / "config.yaml"

# Загружаем конфиг
config = loader.load_config(config_path)

## 5.1. Загрузка данных

In [7]:
# Загрузка train
df_train = loader.data_load_preprocessed(data_type='train', config=config)

[⧗] Загружаю данные из: D:\Skills\Kaggle\ml-regression_concrete-strength\data\processed\eda_data_train.pkl
[✓] Данные успешно загружены. Форма: (781, 11)


In [8]:
# Вывод первых 5 строк тренировочного датасета
df_train.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength,W/C,Sp/C_pct
0,376.0,0.0,0.0,214.6,0.0,1003.5,762.4,3,16.28,0.570745,0.000000
1,491.0,26.0,123.0,210.0,3.9,882.0,699.0,56,59.59,0.427699,0.007943
2,250.0,0.0,95.7,187.4,5.5,956.9,861.2,3,13.82,0.749600,0.022000
3,310.0,0.0,0.0,192.0,0.0,1012.0,830.0,90,35.76,0.619355,0.000000
4,252.1,97.1,75.6,193.8,8.3,835.5,821.4,28,33.40,0.768743,0.032923


In [9]:
# Загрузка test
df_test = loader.data_load_preprocessed(data_type='test', config=config)

[⧗] Загружаю данные из: D:\Skills\Kaggle\ml-regression_concrete-strength\data\processed\eda_data_test.pkl
[✓] Данные успешно загружены. Форма: (228, 10)


In [10]:
# Вывод первых 5 строк тестового датасета
df_test.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,W/C,Sp/C_pct
0,167.4,129.9,128.6,175.5,7.8,1006.3,746.6,28,1.048387,0.046595
1,475.0,118.8,0.0,181.1,8.9,852.1,781.5,7,0.381263,0.018737
2,251.4,0.0,118.3,188.5,6.4,1028.4,757.7,100,0.749801,0.025457
3,307.0,0.0,0.0,193.0,0.0,968.0,812.0,365,0.628664,0.000000
4,143.6,0.0,174.9,158.4,17.9,942.7,844.5,28,1.103064,0.124652
